In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import os
import numpy as np
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import SelectFromModel

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score


In [ ]:
label_encoder = LabelEncoder()
scaler = StandardScaler()

In [ ]:
Parcel = pd.read_csv('/content/FinHack/APNLA.csv')
Burg = pd.read_csv('/content/FinHack/CrimesBurgalry.csv')
Loc = pd.read_csv('/content/FinHack/CrimesLocation.csv')
Soil = pd.read_csv('/content/FinHack/Soil.csv')

In [ ]:
Parcel.head()

,AUTO_ID,PIN,PIND,APN
0,864729,150A221 83,150A221-83,5.464005e+09
1,683969,216B161 104,216B161-104,2.514006e+09
2,1059724,141B185 415,141B185-415,5.533035e+09
3,166893,121-5A223 92,121-5A223-92,5.188025e+09
4,76104,024B193 463,024B193-463,7.444007e+09


In [ ]:
Burg.head()

,key,value
0,Residence - Day,34220.0
1,Non-Residence - Night,21425.0
2,Residence - Unknown,19204.0
3,Residence - Night,18846.0
4,Non-Residence - Day,18334.0


In [ ]:
Loc.head()

,key,value
0,Highway,48459
1,Commercial House,16281
2,Residence,5040
3,Miscellaneous,3873
4,Convenience Store,1717


In [ ]:
Soil.head()

,OBJECTID,AREA_,PERIMETER,CLASS,AREA_ACRES,SHAPEAREA,SHAPELEN
0,1,2.028897e+07,24001.01795,109,466,2.028897e+07,24001.017553
1,2,3.345218e+07,50122.61256,20,768,3.345218e+07,50122.612908
2,3,1.910024e+07,19388.78544,109,438,1.910024e+07,19388.785604
3,4,4.308253e+06,10874.61701,117,99,4.308252e+06,10874.617468
4,5,2.404614e+07,26776.38465,113,552,2.404614e+07,26776.384560


In [ ]:
Parcel.isnull().sum()

,0
AUTO_ID,0
PIN,1
PIND,280
APN,1


In [ ]:
Burg.isnull().sum()

,0
key,0
value,1


In [ ]:
Loc.isnull().sum()

,0
key,0
value,0


In [ ]:
Soil.isnull().sum()

,0
OBJECTID,0
AREA_,0
PERIMETER,0
CLASS,0
AREA_ACRES,0
SHAPEAREA,0
SHAPELEN,0


In [ ]:
Parcel.drop_duplicates()
Parcel.dropna()

,AUTO_ID,PIN,PIND,APN
0,864729,150A221 83,150A221-83,5.464005e+09
1,683969,216B161 104,216B161-104,2.514006e+09
2,1059724,141B185 415,141B185-415,5.533035e+09
3,166893,121-5A223 92,121-5A223-92,5.188025e+09
4,76104,024B193 463,024B193-463,7.444007e+09
...,...,...,...,...
474604,597450,195B145 386,195B145-386,2.654008e+09
474605,485843,177B105 68,177B105-68,2.149005e+09
474606,480017,171B141 144,171B141-144,2.261015e+09
474607,42004,012B193 1130,012B193-1130,7.460021e+09


In [ ]:
Burg.drop_duplicates()
Burg.dropna()

,key,value
0,Residence - Day,34220.0
1,Non-Residence - Night,21425.0
2,Residence - Unknown,19204.0
3,Residence - Night,18846.0
4,Non-Residence - Day,18334.0
5,Non-Residence - Unknown,13308.0


In [ ]:
Parcel = Parcel.reset_index(drop=True)
Burg = Burg.reset_index(drop=True)
Loc = Loc.reset_index(drop=True)
Soil = Soil.reset_index(drop=True)

In [ ]:
combined_df = pd.concat([Parcel, Burg, Loc, Soil], axis=1)
combined_df['Risk Score'] = None
combined_df = combined_df.reset_index(drop=True)
combined_df.head()

,AUTO_ID,PIN,PIND,APN,key,value,key,value,OBJECTID,AREA_,PERIMETER,CLASS,AREA_ACRES,SHAPEAREA,SHAPELEN,Risk Score
0,864729,150A221 83,150A221-83,5.464005e+09,Residence - Day,34220.0,Highway,48459.0,1.0,2.028897e+07,24001.01795,109.0,466.0,2.028897e+07,24001.017553,None
1,683969,216B161 104,216B161-104,2.514006e+09,Non-Residence - Night,21425.0,Commercial House,16281.0,2.0,3.345218e+07,50122.61256,20.0,768.0,3.345218e+07,50122.612908,None
2,1059724,141B185 415,141B185-415,5.533035e+09,Residence - Unknown,19204.0,Residence,5040.0,3.0,1.910024e+07,19388.78544,109.0,438.0,1.910024e+07,19388.785604,None
3,166893,121-5A223 92,121-5A223-92,5.188025e+09,Residence - Night,18846.0,Miscellaneous,3873.0,4.0,4.308253e+06,10874.61701,117.0,99.0,4.308252e+06,10874.617468,None
4,76104,024B193 463,024B193-463,7.444007e+09,Non-Residence - Day,18334.0,Convenience Store,1717.0,5.0,2.404614e+07,26776.38465,113.0,552.0,2.404614e+07,26776.384560,None


In [ ]:
# Download training data from open datasets.
train_data = combined_df.drop(['PIN', 'PIND','Risk Score', 'key', 'key'], axis=1)

# Download test data from open datasets.
test_data = combined_df['Risk Score']

In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10, 512),  # Changed input size
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=10, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
train_data.dtypes

,0
AUTO_ID,int64
APN,float64
value,float64
value,float64
OBJECTID,float64
AREA_,float64
PERIMETER,float64
CLASS,float64
AREA_ACRES,float64
SHAPEAREA,float64


In [ ]:
test_data.dtypes

dtype('O')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')  # or sparse=False
train_labels = encoder.fit_transform(train_data.values.reshape(-1, 1))
test_labels = encoder.transform(test_data.values.reshape(-1, 1))  # test_data is 'Risk Score' column


In [ ]:

train_data_tensor = torch.tensor(train_data.values, dtype=torch.float32)
#test_data_tensor = torch.tensor(test_data.values, dtype=torch.float32)
test_data_tensor = torch.tensor(test_data.values.astype(np.float32), dtype=torch.float32)


In [ ]:
train_dataloader = DataLoader(train_data_tensor, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data_tensor, batch_size=64, shuffle=False)

In [ ]:

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer) # Use train_dataloader
    test(test_dataloader, model, loss_fn) # Use test_dataloader
print("Done!")

Epoch 1
-------------------------------


ValueError: too many values to unpack (expected 2)